Train on core terms and consequences

In [4]:
import pickle
import numpy as np
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib
from keras.callbacks import Callback
from sklearn.preprocessing import LabelEncoder

class F1ScoreCallback(Callback):
    def __init__(self, X_val, y_val):
        super(F1ScoreCallback, self).__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.best_f1 = 0.0
        self.best_model = None
        self.f1_scores = []

    def on_epoch_end(self, epoch, logs=None):
        y_val_pred = np.argmax(self.model.predict(self.X_val), axis=1)
        f1 = f1_score(self.y_val, y_val_pred, average='weighted')
        self.f1_scores.append(f1)
        

        if f1 > self.best_f1:
            self.best_f1 = f1
            self.best_model = self.model
            print(f"Epoch {epoch + 1} - F1 Score: {f1:.4f}")
            print("Saved best model")
            print(self.f1_scores)

with open('train_without_test4.pickle', 'rb') as f1:
    balanced = pickle.load(f1)

with open('test_core_cons_terms_with_embeddings.pickle', 'rb') as f2:
    unbalanced = pickle.load(f2)

train = np.array([item['cve_core_consequences_ada_embedding'] for item in balanced if item['cwe'] != 'None'])
test = np.array([item['cwe'] for item in balanced if item['cwe'] != 'None'])
np.random.seed(42)
X_train, X_val, y_train, y_val = train_test_split(train,test,test_size=0.1,random_state=42)

X_test = np.array([item['cve_core_consequences_ada_embedding'] for item in unbalanced if item['cwe'] != 'None'])
y_test = np.array([item['cwe'] for item in unbalanced if item['cwe'] != 'None'])

label_encoder_train = LabelEncoder()
y_train_encoded = label_encoder_train.fit_transform(y_train)
label_encoder_test = LabelEncoder()
y_test_encoded = label_encoder_test.fit_transform(y_test)


input_dim = X_train.shape[1]
output_dim = len(np.unique(y_train))

model = Sequential()
model.add(Dense(128, input_dim=input_dim, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(output_dim, activation='softmax'))


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

f1_callback = F1ScoreCallback(X_val, label_encoder_train.transform(y_val))

history = model.fit(X_train, y_train_encoded, epochs=40, batch_size=32, validation_data=(X_val, label_encoder_train.transform(y_val)), verbose=1, callbacks=[f1_callback])

best_model = f1_callback.best_model


# Save the best model
joblib.dump(best_model, 'best_model.joblib')

# Make predictions on the test set
y_pred_probs = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

y_pred_original = label_encoder_train.inverse_transform(y_pred)

print("Classification Report:\n", classification_report(y_test, y_pred_original, digits=4))

joblib.dump(label_encoder_train, 'label_encoder_train.joblib')

Epoch 1/40
260/260 [==============================] - 0s 1ms/step loss: 1.9024 - acc
Epoch 1 - F1 Score: 0.5708
Saved best model
[0.5707787828566004]
2336/2336 [==============================] - 8s 3ms/step - loss: 1.9018 - accuracy: 0.4944 - val_loss: 1.4796 - val_accuracy: 0.5909
Epoch 2/40
260/260 [==============================] - 0s 1ms/step loss: 1.3675 - accu
Epoch 2 - F1 Score: 0.6063
Saved best model
[0.5707787828566004, 0.6063319005153387]
2336/2336 [==============================] - 7s 3ms/step - loss: 1.3673 - accuracy: 0.6166 - val_loss: 1.3482 - val_accuracy: 0.6248
Epoch 3/40
260/260 [==============================] - 0s 1ms/step loss: 1.2747 - accu
Epoch 3 - F1 Score: 0.6189
Saved best model
[0.5707787828566004, 0.6063319005153387, 0.6189369599605967]
2336/2336 [==============================] - 7s 3ms/step - loss: 1.2746 - accuracy: 0.6396 - val_loss: 1.3133 - val_accuracy: 0.6319
Epoch 4/40
260/260 [==============================] - 1s 2ms/step loss: 1.222
Epoch 4 - F

['label_encoder_train.joblib']

Run inference

In [6]:
import pickle
import numpy as np
from sklearn.metrics import classification_report
import joblib
from sklearn.preprocessing import LabelEncoder
# Save the best model
with open('test_core_cons_terms_with_embeddings.pickle', 'rb') as f2:
    unbalanced = pickle.load(f2)

X_test = np.array([item['cve_core_consequences_ada_embedding'] for item in unbalanced if item['cwe'] != 'None'])
y_test = np.array([item['cwe'] for item in unbalanced if item['cwe'] != 'None'])

best_model=joblib.load('best_model.joblib')
label_encoder_train=joblib.load('label_encoder_train.joblib')

# Make predictions on the test set
y_pred_probs = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

y_pred_original = label_encoder_train.inverse_transform(y_pred)

print("Classification Report:\n", classification_report(y_test, y_pred_original, digits=4))

444/444 [==============================] - 1s 2ms/step
Classification Report:
               precision    recall  f1-score   support

         119     0.8061    0.4467    0.5749      1070
         120     0.3931    0.6378    0.4864       196
         125     0.7504    0.8759    0.8083       532
         134     0.6429    0.9474    0.7660        19
         189     0.6937    0.6471    0.6696       119
         190     0.7071    0.8450    0.7699       200
          20     0.4440    0.2790    0.3427       810
         200     0.5479    0.5136    0.5302       590
         203     0.6000    0.7778    0.6774        27
          22     0.8619    0.8919    0.8767       518
         254     0.0769    0.1176    0.0930        34
         255     0.2941    0.2239    0.2542        67
         264     0.5051    0.3956    0.4437       503
         269     0.2432    0.5094    0.3293       106
         276     0.2738    0.3594    0.3108        64
         284     0.2368    0.1463    0.1809       123
  